#### Installs

In [ ]:
%%capture
!pip install pandas-profiling
!pip install arabic-reshaper
!pip install python-bidi
!pip install folium

#### Imports

In [ ]:
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from pandas_profiling import ProfileReport
import folium  # gelocation plotting
from scipy import stats
from folium.plugins import MarkerCluster
import arabic_reshaper  # arabic plotting
from bidi.algorithm import get_display  # arabic encoding
from sklearn.cluster import KMeans  # clustering
from sklearn import preprocessing

#### Conf

In [ ]:
sns.set(rc={"figure.figsize": (11.7, 8.27)})
sns.set_style("whitegrid")
colors = sns.color_palette("pastel")[0:5]
warnings.filterwarnings("ignore")

#### Data Loading

In [ ]:
datasets = ["customers.csv", "trucks.csv", "orders.csv", "items.csv"]
base_dir = "datasets/"

customers_df = pd.read_csv(f"{base_dir}{datasets[0]}")
trucks_df = pd.read_csv(f"{base_dir}{datasets[1]}")
orders_df = pd.read_csv(f"{base_dir}{datasets[2]}")
items_df = pd.read_csv(f"{base_dir}{datasets[3]}")

#### Data Transformation

In [ ]:
# cast to datetime
customers_df["registration_date"] = pd.to_datetime(customers_df["registration_date"])
customers_df["last_order_date"] = pd.to_datetime(customers_df["last_order_date"])
orders_df["Order_date"] = pd.to_datetime(orders_df["Order_date"])

### EDA

#### Customers Data

In [ ]:
ProfileReport(customers_df, title="Customers Profiling")
# profile.to_file("customers_report.html") #save report

From the above report, we can conclude:
- Customers data covers registered 9810 user in a year and 2 months **(12-12-2019 to 04-01-2021)**
- User registration peak was on June 2020 (3500)
- Customers activity peak was on Feburary and March 2021, maybe due to campaigns/items seasonality
- Customer inactivity peak was in November and early January 2021
- `Others` imputation is highly used in `Arabic Area Name` (~11%), which can be treated by geo-approximation to find nearest areas

- There's slight positive correlation between `Canceled Transactions` and `Average Ticket Size`

- The nullity correlation states that the abscence of any purchasing feature (seetting an order, cancelling an order, average order size, etc..) affects the other features. 

- An inactive customer is someone with Nullified `Number of Transactions`

In [ ]:
cairo = ["30.0444", "31.2357"]
customers_map = folium.Map(cairo, zoom_start=8)
marker_cluster = MarkerCluster().add_to(customers_map)

loc = (
    customers_df[["buyer_business_type", "geo_latitude", "geo_longitude"]]
    .dropna()
    .reset_index()
)
for i in range(len(loc)):
    folium.Marker(
        [loc["geo_latitude"][i], loc["geo_longitude"][i]],
        popup="<i>Customer {0},{1}</i>".format(loc["geo_latitude"][i], loc["geo_longitude"][i]),
        tooltip=loc["buyer_business_type"][i],
    ).add_to(marker_cluster)

customers_map

- The Map shows lack of coverage in active economic areas, such as New Cairo, Zayed, and 6th October
- There's messy points in geolocation, like there's a customer in Libya and another in Europe

In [ ]:
registration_dates = customers_df["registration_date"].astype(str).str.rstrip()
date_counts = registration_dates.value_counts().sort_index()

step = 7
ax = sns.lineplot(x=date_counts.index[::step], y=date_counts.values[::step])
xticks = pd.Series(date_counts.index[::step])
ax.set_xticklabels(xticks, rotation=45)

ax.set_title("Registration Dates", fontsize=18)
ax.set_xlabel("Date", fontsize=15)
ax.set_ylabel("Count", fontsize=15)
plt.show()

- Registeration dates spiked in May and June 2020, Which is either a result by first-day promo or seasonality correlation
- Registeration rate has been decreased dramiltically in the rest of Summer till End of November
- The data team should invistigate such correlation extensively to mimic the same behavior, thus attracting new customers

#### Orders Data

In [ ]:
orders_df["order_date_str"] = orders_df["Order_date"].astype(str).str.rstrip()
ax = sns.countplot(
    x=orders_df["order_date_str"],
    data=orders_df,
    order=orders_df["order_date_str"].value_counts().index,
)
ax.set_title("Order Dates Frequency", fontsize=18)
ax.set_xlabel("Order Date", fontsize=15)
ax.set_ylabel("Count", fontsize=15)
plt.tight_layout()

- Orders huge spike on the 22th Of September 2020
- Due to orders limitation, no real insights could be drawn from last figure

In [ ]:
customers_orders_df = pd.merge(customers_df, orders_df, on="customer_id")

In [ ]:
valid_orders_dates = (
    customers_orders_df["Order_date"] > customers_orders_df["registration_date"]
)

ax = plt.pie(
    valid_orders_dates.value_counts(),
    labels=valid_orders_dates.value_counts().index,
    autopct="%.0f%%",
    colors=colors,
)
plt.title("Valid Order Dates", fontsize=18)

Order Date Validation methodology is calculated as the following: 
- If the order precedes the customer's registration date, the order is considered `Invalid`
- Else, the order is considered `Valid`

In [ ]:
customers_orders_df = customers_orders_df.replace(
    {"شبرا الخيمة": "شبرا الخيمه"}
)  # area name unification

In [ ]:
# arabic characters encoding and correction
districts_orders_amount = customers_orders_df.groupby("district_name_ar").sum()[
    "total_orders_amount"
]
district_orders_amount_corrected = {
    get_display(arabic_reshaper.reshape(k)): v
    for k, v in districts_orders_amount.to_dict().items()
}
districts_orders_amount_df = pd.DataFrame(
    district_orders_amount_corrected.items(), columns=["area", "order_amount"]
)

In [ ]:
ax = sns.barplot(
    x="area",
    y="order_amount",
    data=districts_orders_amount_df,
    order=districts_orders_amount_df.sort_values("order_amount", ascending=False).area,
)  # sort areas by order amount

xticks = districts_orders_amount_df.area.values
ax.set_xticks(range(0, len(xticks)))  # set ticks positions
ax.set_xticklabels(xticks, rotation=45)  # set ticks labels (areas names)

ax.set_title("Areas vs Order Amounts (to the million)", fontsize=18)
ax.set_xlabel("Area", fontsize=15)
ax.set_ylabel("Order Amount", fontsize=15)
plt.show()

- The above figure shows orders size by area $ x10^{6} $ 
- The first 4 districts account for +90% of the total order amount

In [ ]:
orders_items_df = pd.merge(orders_df, items_df, on="item_id")
items_order_freq = orders_items_df.groupby("item_id").sum()["requested_quantity"]
sorted_items_order_freq = items_order_freq.sort_values(ascending=False)[:20]

In [ ]:
ax = sns.barplot(
    x=sorted_items_order_freq.index,
    y=sorted_items_order_freq.values,
    order=sorted_items_order_freq.sort_values(ascending=False).index,
)  # sort areas by order amount

ax.set_title("Items Ordered Quantity", fontsize=18)
ax.set_xlabel("Item ID", fontsize=15)
ax.set_ylabel("Quantity", fontsize=15)
plt.show()

- The above figure shows the top 20 requested items 
- This needs more interpetation using the items features 

#### Items Data

In [ ]:
items_df["vol"] = items_df.length.multiply(items_df.width).multiply(items_df.height)
items_df["vol"].hist()
plt.title("Items Volume Histogram")

- The items volume distribution is left skewed, which means most of the items considered to be small sized 

In [ ]:
pd.merge(sorted_items_order_freq, items_df, on="item_id").drop_duplicates()[
    "vol"
].hist()
plt.title("Highest Demand Items Volume")

- The Items with the highest Demand, are considered to be small in size as well 

#### Trucks Data

As trucks talbe is non-connected floating table, it won't be insightful to conduct analysis for 

### Customers Clustering

In [ ]:
customers_df.info()

In [ ]:
# Label encoding
le = preprocessing.LabelEncoder()
customers_df["buyer_business_type_encoded"] = le.fit_transform(
    customers_df["buyer_business_type"]
)

In [ ]:
# remove outliers
features = [
    "buyer_business_type_encoded",
    "number_of_transactions",
    "total_orders_amount",
]
customers_df.dropna(subset = features, inplace = True)
customers_df = customers_df[
    (np.abs(stats.zscore(customers_df[features])) < 3).all(axis=1)
]
X = customers_df[features]

In [ ]:
# decide k using elbow method
wcss = []
for i in range(1, 11):
    km = KMeans(n_clusters=i)
    km.fit(X)
    wcss.append(km.inertia_)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(range(1, 11), wcss)
plt.plot(range(1, 11), wcss, linewidth=2, color="red", marker="8")
plt.xlabel("K Value")
plt.xticks(np.arange(1, 11, 1))
plt.ylabel("WCSS")
plt.show()

- Suitable Number of Clusters is **4**

In [ ]:
# taking 4 clusters
km = KMeans(n_clusters=4)
km.fit(X)
y = km.predict(X)
customers_df["label"] = y
customers_df.head()

In [ ]:
ax = sns.scatterplot(
    x="buyer_business_type",
    y="total_orders_amount",
    hue="label",
    palette=["green", "orange", "brown", "dodgerblue"],
    legend="full",
    data=customers_df,
    s=60,
)
ax.set_title("Busniesses Types and Spending Behavior", fontsize=18)
ax.set_xlabel("Business Type", fontsize=15)
ax.set_ylabel("Total Order Amount", fontsize=15)
plt.show()

- The spending behaviour of 'اخري' clusters is similar to 'كشك' ones
- We can presume such assumption and replace the 'اخري' values

In [ ]:
customers_df["buyer_business_type"].replace({"اخري": "كشك"}, inplace=True)

Setting Customer Promotion:
I suggest offering discounts relative with the order size (to encourage the customer to move to the next spending layer)
- Label 1 => +5000 spendings (10% off)
- Label 0 => +10000 spendings (15% off)
- Label 3 => +15000 spendings (18% off)
- Label 2 => +17000 spendings (20% off)

In [ ]:
promo_dict = {
    1: "+5000 spendings (10% off)",
    0: "+10000 spendings (15% off)",
    3: "+15000 spendings (18% off)",
    2: "+17000 spendings (20% off)",
}

promos = []
customers_df.rename({"label": "cluster"}, axis=1, inplace=True)
for cluster in customers_df["cluster"]:
    promos.append(promo_dict[cluster])
customers_df["promo"] = promos

In [ ]:
customers_df.to_csv(f'{base_dir}customers_clustered.csv')